## Homework 10: GPUs

## Due Date: May 04, 2020, 08:00am

#### Firstname Lastname: 

#### E-mail: 

#### Enter your solutions and submit this notebook

---

**Problem 1 (100p)**


Use PyOpenCL to write a program which will be able to run in parallel on GPUs. 


Your program will:

- draw two random vectors $\vec u$ and $\vec v$ from $[0,1]^N$ where $N = 10^7$;


- calculate and output similarity between $\vec u$ and $\vec v$.




The similarity between two vectors $\vec u$ and $\vec v$ is defined here as a `cosine` value of the angle between them $\measuredangle \left( \vec u, \vec v \right)$. That is, the program returns: 

$$\cos \left( \measuredangle \left( \vec u, \vec v \right) \right)$$.


Note that the output is a real value and must belong to $[-1, 1]$.

In [1]:
!export NUMBA_ENABLE_CUDASIM=1

We use the following relation that `cosine`of the angle between two vectors is given by the ratio
of their inner product and the product of its intensities: 

$$\frac{(\vec u, \vec v)}{|\vec u||\vec v|}.$$

               

In [4]:
# the same above algorithm but written in a different way
from time import time
import numpy as np
import pyopencl as cl


n = 10_000_000
time_start = time()

# generate random vectors
u = np.random.rand(n).astype(np.float32)
v = np.random.rand(n).astype(np.float32)

ctx   = cl.create_some_context()
queue = cl.CommandQueue(ctx)

mf = cl.mem_flags
u_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=u)
v_g = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=v)

prg = cl.Program(ctx, """
__kernel void fnct(
__global const float *u_g, 
__global const float *v_g, 
__global float *uu_g,
__global float *vv_g,
__global float *uv_g){

int gid = get_global_id(0);
uu_g[gid] = u_g[gid] * u_g[gid];
vv_g[gid] = v_g[gid] * v_g[gid];
uv_g[gid] = u_g[gid] * v_g[gid];
}""").build()

uu_g = cl.Buffer(ctx, mf.WRITE_ONLY, u.nbytes)
vv_g = cl.Buffer(ctx, mf.WRITE_ONLY, u.nbytes)
uv_g = cl.Buffer(ctx, mf.WRITE_ONLY, u.nbytes)

prg.fnct(queue, u.shape, None, u_g, v_g, uu_g, vv_g, uv_g)

uu = np.empty_like(u)
vv = np.empty_like(u)
uv = np.empty_like(u)

cl.enqueue_copy(queue, uu, uu_g)
cl.enqueue_copy(queue, vv, vv_g)
cl.enqueue_copy(queue, uv, uv_g)

cosine_value = sum(uv) / np.sqrt(np.sum(uu) * np.sum(vv))

print("time_in_secs: %0.8f" % (time() - time_start))
print("cosine_value: %0.8f" % cosine_value)

time_in_secs: 2.17776132
cosine_value: 0.75007637
